In [1]:
# Import libraries and set desired options

from __future__ import division, print_function
# Disable Anaconda warnings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

##### Approaching the Problem
We will be solving the intruder detection problem analyzing his behavior on the Internet. It is a complicated and interesting problem combining the data analysis and behavioral psychology.

For example: Yandex solves the mailbox intruder detection problem based on the user's behavior patterns. In a nutshell, intruder's behaviour pattern might differ from the owner's one: 
- the breaker might not delete emails right after they are read, as the mailbox owner might do
- the intruder might mark emails and even move the cursor differently
- etc.

So the intruder could be detected and thrown out from the mailbox proposing the owner to be authentificated via SMS-code.
This pilot project is described in the Habrahabr article.

Similar things are being developed in Google Analytics and described in scientific researches. You can find more on this topic by searching "Traversal Pattern Mining" and "Sequential Pattern Mining".

In this competition we are going to solve a similar problem: our algorithm is supposed to analyze the sequence of websites consequently visited by a particular person and to predict whether this person is Alice or an intruder (someone else). As a metric we will use [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). We will reveal who Alice is at the end of the course.

### 1. Data Downloading and Transformation
Register on [Kaggle](www.kaggle.com), if you have not done it before.
Go to the competition [page](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) and download the data.

First, read the training and test sets. Then explore the data and perform a couple of simple exercises:

In [2]:
# Read the training and test data sets
PATH_TO_DATA = ('/mnt/Storage/Programming/ml_course/mlcourse_open/jupyter_russian/homeworks/hw6_data')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id')

# Switch time1, ..., time10 columns to datetime type
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


The training data set contains the following features:

- **site1** – id of the first visited website in the session
- **time1** – visiting time for the first website in the session
- ...
- **site10** – id of the tenth visited website in the session
- **time10** – visiting time for the tenth website in the session
- **target** – target variable, possesses value of 1 for Alice's sessions, and 0 for the other users' sessions
    
User sessions are chosen in the way they are not longer than half an hour or/and contain more than ten websites. I.e. a session is considered as ended either if a user has visited ten websites or if a session has lasted over thirty minutes.

There are some empty values in the table, it means that some sessions contain less than ten websites. Replace empty values with 0 and change columns types to integer. Also load the websites dictionary and check how it looks like:

In [3]:
# Change site1, ..., site10 columns type to integer and fill NA-values with zeros
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# Load websites dictionary
with open(os.path.join(PATH_TO_DATA, r"site_dic.pkl"), "rb") as input_file:
    site_dict = pickle.load(input_file)

# Create dataframe for the dictionary
sites_dict = pd.DataFrame(list(site_dict.keys()), index=list(site_dict.values()), columns=['site'])
print(u'Websites total:', sites_dict.shape[0])
sites_dict.head()

Websites total: 48371


,site
41172,d2xx6zcy9l0x7e.cloudfront.net
30101,www.antredesyria.com
3897,fr.clickintext.net
6864,profs.cmaisonneuve.qc.ca
456,www.lexpress.fr


In [4]:
# Answer
print(test_df.shape, train_df.shape)

(82797, 20) (253561, 21)


In [5]:
# Our target variable
y_train = train_df['target']

# United dataframe of the initial data 
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# Index to split the training and test data sets
idx_split = train_df.shape[0]

full_df.shape

(336358, 20)

For the very basic model, we will use only the visited websites in the session (but we will not take into account timestamp features). The point behind this data selection is: *Alice has her favorite sites, and the more often you see these sites in the session, the higher probability that this is an Alice's session, and vice versa.*

Let us prepare the data, we will take only features `site1, site2, ... , site10` from the whole dataframe. Keep in mind that the missing values are replaced with zero. Here is how the first rows of the dataframe look like:

In [6]:
# Dataframe with indices of visited websites in session
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Sessions are the sequences of website indices, and data in this representation is inconvenient for linear methods. According to our hypothesis (Alice has favorite websites) we need to transform this dataframe so each website has corresponding feature (column) and its value is equal to number of this website visits in the session. It can be done in two lines:

In [7]:
# sequence of indices
sites_flatten = full_sites.values.flatten()

# and the matrix we are looking for
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0]  + 10, 10)))[:, 1:]

# print(full_sites_sparse[:, :100])

In [8]:
import datetime as dt

times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

In [9]:
full_df['hour'] = full_df['time1'].dt.hour
full_df['day'] = full_df['time1'].dt.weekday
full_df['day_of_year'] = full_df['time1'].dt.day
full_df['week'] = full_df['time1'].dt.week
full_df['month'] = full_df['time1'].dt.month
full_df['year'] = full_df['time1'].dt.year - 2012

In [10]:
def get_time_zone(hour):
    if hour >= 7 and hour<10:
        return 1
    elif hour >= 10 and hour<14:
        return 2
    elif hour >= 14 and hour<=23:
        return 3

In [11]:
full_df['time_zone'] = full_df['hour'].apply(get_time_zone)
full_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time9,site10,time10,hour,day,day_of_year,week,month,year,time_zone
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,8,5,12,2,1,1,1
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,NaT,0,NaT,8,5,12,2,1,1,1
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,2013-01-12 08:50:17,946,2013-01-12 08:50:17,8,5,12,2,1,1,1
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,2013-01-12 08:50:19,946,2013-01-12 08:50:20,8,5,12,2,1,1,1
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,2013-01-12 08:50:22,947,2013-01-12 08:50:22,8,5,12,2,1,1,1
242171,952,2013-01-12 08:50:22,947,2013-01-12 08:50:23,953,2013-01-12 08:50:23,946,2013-01-12 08:50:23,947,2013-01-12 08:50:24,...,2013-01-12 08:50:25,947,2013-01-12 08:50:25,8,5,12,2,1,1,1
57157,953,2013-01-12 08:50:25,947,2013-01-12 08:50:26,946,2013-01-12 08:50:26,953,2013-01-12 08:50:26,955,2013-01-12 08:50:26,...,2013-01-12 08:50:28,1033,2013-01-12 08:50:28,8,5,12,2,1,1,1
240201,946,2013-01-12 08:50:28,947,2013-01-12 08:50:28,954,2013-01-12 08:50:28,953,2013-01-12 08:50:29,946,2013-01-12 08:50:29,...,2013-01-12 08:50:31,956,2013-01-12 08:50:31,8,5,12,2,1,1,1
210686,946,2013-01-12 08:50:31,956,2013-01-12 08:50:32,946,2013-01-12 08:50:32,946,2013-01-12 08:50:33,955,2013-01-12 08:50:33,...,2013-01-12 08:50:36,948,2013-01-12 08:50:36,8,5,12,2,1,1,1


In [13]:
from collections import Counter
def get_number_of_duplicates(row):
    counter_dict = dict(Counter(row.tolist()))
    counter_dict = {k:v for k,v in counter_dict.items() if k!=0}
    max_duplicates = max(counter_dict.values())
    return max_duplicates

def get_mean_time_between_sites(row):
    timedeltas = []
    time_1 = row[row.index[1]]
    
    for col in row.index[1:]:
        time_2 = row[col]
        if not (type(time_2) is pd._libs.tslib.NaTType):
            timedeltas.append((time_2 - time_1).seconds)
            time_1 = row[col]
        else:
            continue
    if len(timedeltas) == 0:
        return 0
    return sum(timedeltas)/len(timedeltas)


def get_mean_time_between_duplicates(row):
    sites = {}
    for i in range(1,11):
        site = row['site{}'.format(str(i))]
        time = row['time{}'.format(str(i))]
        if site==0:
            break
        if not sites.get(site):
            sites[site] = time
        else:
            sites[site] = (time - sites[site])
    times = [value.seconds for value in sites.values() if type(value) is pd._libs.tslib.Timedelta] 
    return pd.Series(times).mean()

In [28]:
# full_df['mean_time_between_duplicates'] = full_df.apply(get_mean_time_between_duplicates, axis=1)
# full_df['max_duplicates'] = full_df[sites].apply(get_number_of_duplicates, axis=1)

In [14]:
full_df['mean_time_between_sessions'] = full_df[times].apply(get_mean_time_between_sites, axis=1)

In [37]:
full_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,site10,time10,hour,day,week,month,year,time_zone,max_duplicates,mean_time_between_sessions
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,0,NaT,8,5,2,1,1,1,1,0.000000
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,0,NaT,8,5,2,1,1,1,2,595.333333
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,946,2013-01-12 08:50:17,8,5,2,1,1,1,5,0.333333
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,946,2013-01-12 08:50:20,8,5,2,1,1,1,3,0.333333
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,947,2013-01-12 08:50:22,8,5,2,1,1,1,3,0.222222
242171,952,2013-01-12 08:50:22,947,2013-01-12 08:50:23,953,2013-01-12 08:50:23,946,2013-01-12 08:50:23,947,2013-01-12 08:50:24,...,947,2013-01-12 08:50:25,8,5,2,1,1,1,3,0.222222
57157,953,2013-01-12 08:50:25,947,2013-01-12 08:50:26,946,2013-01-12 08:50:26,953,2013-01-12 08:50:26,955,2013-01-12 08:50:26,...,1033,2013-01-12 08:50:28,8,5,2,1,1,1,4,0.222222
240201,946,2013-01-12 08:50:28,947,2013-01-12 08:50:28,954,2013-01-12 08:50:28,953,2013-01-12 08:50:29,946,2013-01-12 08:50:29,...,956,2013-01-12 08:50:31,8,5,2,1,1,1,3,0.333333
210686,946,2013-01-12 08:50:31,956,2013-01-12 08:50:32,946,2013-01-12 08:50:32,946,2013-01-12 08:50:33,955,2013-01-12 08:50:33,...,948,2013-01-12 08:50:36,8,5,2,1,1,1,6,0.444444


In [16]:
time_zone_dummies = pd.get_dummies(full_df['time_zone'], prefix='time_zone')
hour_dummies = pd.get_dummies(full_df['hour'], prefix='hour')
day_dummies = pd.get_dummies(full_df['day'], prefix='day')
week_dummies = pd.get_dummies(full_df['week'], prefix='week')
month_dummies = pd.get_dummies(full_df['month'], prefix='month')
year_dummies = pd.get_dummies(full_df['year'], prefix='year')
day_of_year_dummies = pd.get_dummies(full_df['day_of_year'], prefix='day_of_year')
# max_duplicates_dummies = pd.get_dummies(full_df['max_duplicates'], prefix='max_duplicates')

In [34]:
full_sparse = csr_matrix(hstack([full_sites_sparse, week_dummies, day_dummies, hour_dummies]))
print(full_sparse.shape)

(336358, 48437)


### 3. Training the first model

So, we have an algorithm and data for it. Let us build our first model, using [logistic regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) implementation from ` Sklearn` with default parameters. We will use the first 90% of the data for training (the training data set is sorted by time), and the remaining 10% for validation. Let's write a simple function that returns the quality of the model and then train our first classifier:

In [35]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
def get_auc_lr_valid(X, y, C=1.0, seed=np.random.randint(100), ratio = 0.9):
    # Split the data into the training and validation sets
    idx = int(round(X.shape[0] * ratio))
    # Classifier training
#     lr = LogisticRegression(C=C, random_state=seed, n_jobs=-1).fit(X[:idx, :], y[:idx])
    lr = LogisticRegression(random_state=seed, n_jobs=-1)
    logit_pipe_params = {'C': np.logspace(-4, 4, 9)}
    logit_pipe_grid_search = GridSearchCV(lr, logit_pipe_params, n_jobs=-1, scoring ='roc_auc')
    logit_pipe_grid_search.fit(X[:idx, :], y[:idx])
    # Prediction for validation set
    y_pred = logit_pipe_grid_search.predict_proba(X[idx:, :])[:, 1]
    print(logit_pipe_grid_search.best_params_)
    # Calculate the quality
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [ ]:
%%time
# Select the training set from the united dataframe (where we have the answers)
X_train = full_sparse[:idx_split, :]
print(X_train.shape)

# Calculate metric on the validation set
print(get_auc_lr_valid(X_train, y_train))

(253561, 48437)


The first model demonstrated the quality  of 0.91952 on the validation set. Let's take it as the first baseline and starting point. To make a prediction on the test data set ** we need to train the model again on the entire training data set ** (until this moment, our model used only part of the data for training), which will increase its generalizing ability:

In [32]:
# Function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [33]:
# Train the model on the whole training data set
# Use random_state=17 for repeatability
# Parameter C=1 by default, but here we set it explicitly
lr = LogisticRegression(C=0.1, random_state=17).fit(X_train, y_train)

# Make a prediction for test data set
X_test = full_sparse[idx_split:,:]
y_test = lr.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted
name = '5'
write_to_submission_file(y_test, '/mnt/Storage/Programming/ml_course/mlcourse_open/jupyter_russian/homeworks/submisions/alice/%s.csv' % name)